In [ ]:
%load_ext autoreload
%autoreload 2
!python -m pip install eth_account eth_typing web3

In [64]:
import json
from pathlib import Path
from pprint import pprint
from typing import Any, Optional

from eth_account import Account
from eth_account.signers.local import LocalAccount
from eth_typing import ChecksumAddress
from web3 import AsyncWeb3
from web3.contract.async_contract import AsyncContract, AsyncContractFunction
from web3.middleware import ExtraDataToPOAMiddleware
from web3.types import TxParams, TxReceipt

In [65]:
def load_abi(path: Path) -> Any:
    with open(path) as abi:
        return json.load(abi)


w3 = AsyncWeb3(AsyncWeb3.AsyncHTTPProvider("https://polygon-rpc.com/"))
w3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)
chain_name = "polygon"

abi_path = Path("abi")
erc20_abi = load_abi(abi_path / "ethereum" / "erc20.json")
pool_abi = load_abi(abi_path / "aave" / "pool" / f"{chain_name}.json")

token_address = "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063"  # Polygon DAI
token_contract = w3.eth.contract(address=token_address, abi=erc20_abi)  # type: ignore

atoken_address = "0x82E64f49Ed5EC1bC6e43DAD4FC8Af9bb3A2312EE"  # Polygon aDAI
atoken_contract = w3.eth.contract(address=atoken_address, abi=erc20_abi)  # type: ignore

pool_address = "0x794a61358D6845594F94dc1DB02A252b5b4814aD"  # V3-mainnet Polygon pool
pool_contract = w3.eth.contract(address=pool_address, abi=pool_abi)  # type: ignore

account: LocalAccount = Account.from_key(
    "dcfb4ae8d01174fb61c304b1a9f099437c2a91a9fe698a2fffd358e46d1442a5"
)  # Insert your private key here

solidity_uint_max = 2**256 - 1

In [73]:
async def get_balance(token_contract: AsyncContract, address: ChecksumAddress) -> int:
    return await token_contract.functions.balanceOf(address).call()


first = True  # Whether this is the first transaction


async def send_transaction(
    function: AsyncContractFunction, account: LocalAccount
) -> tuple[TxParams, TxReceipt]:
    global first
    params: TxParams = {
        "from": account.address,
        "nonce": await function.w3.eth.get_transaction_count(
            account.address, "latest" if first else "pending"
        ),
    }
    first = False

    tx = await function.build_transaction(params)
    tx["gas"] *= 2  # type: ignore

    signed = account.sign_transaction(tx)  # type: ignore
    tx_hash = await function.w3.eth.send_raw_transaction(signed.raw_transaction)
    receipt = await function.w3.eth.wait_for_transaction_receipt(tx_hash)

    return (tx, receipt)


withdraws: list[Optional[tuple[TxParams, TxReceipt]]] = []
approves = supplies = withdraws

while True:
    if (atoken_balance := await get_balance(atoken_contract, account.address)) > 0:
        print(f"Withdrawing {atoken_balance} units")
        withdraw_function = pool_contract.functions.withdraw(
            token_address,  # asset
            solidity_uint_max,  # amount
            account.address,  # to
        )
        (params, receipt) = await send_transaction(withdraw_function, account)
        withdraws.append((params, receipt))

        assert receipt["status"] == 0x1, "Transaction failed"
        token_balance = await get_balance(token_contract, account.address)
        atoken_balance = await get_balance(atoken_contract, account.address)
        print(f"Token balance: {token_balance}, aToken balance: {atoken_balance}")
        assert (
            token_balance > 0
        ), "Token balance should be greater than 0 after withdrawing"
    else:
        print(f"Not withdrawing {atoken_balance} units")
        withdraws.append(None)

    if (token_balance := await get_balance(token_contract, account.address)) > 0:
        print(f"Approving {solidity_uint_max} units")
        approve_function = token_contract.functions.approve(
            pool_address, solidity_uint_max
        )
        (params, receipt) = await send_transaction(approve_function, account)
        supplies.append((params, receipt))
        assert receipt["status"] == 0x1, "Transaction failed"

        print(f"Supplying {token_balance} units")
        supply_function = pool_contract.functions.supply(
            token_address,  # asset
            token_balance,  # amount
            account.address,  # onBehalfOf
            0,  # referralCode
        )
        (params, receipt) = await send_transaction(supply_function, account)
        supplies.append((params, receipt))

        assert receipt["status"] == 0x1, "Transaction failed"
        token_balance = await get_balance(token_contract, account.address)
        atoken_balance = await get_balance(atoken_contract, account.address)
        print(f"Token balance: {token_balance}, aToken balance: {atoken_balance}")
        assert token_balance == 0, f"Token balance should be 0 after supplying"
    else:
        print(f"Not supplying {token_balance} units")
        approves.append(None)
        supplies.append(None)

    print()

Not withdrawing 0 units
Approving 115792089237316195423570985008687907853269984665640564039457584007913129639935 units
Supplying 36415129233687899449 units
Token balance: 36415129233687899449, aToken balance: 0


AssertionError: Token balance should be 0 after supplying

In [75]:
# index = 0

# for (withdraw, approve, supply) in zip(withdraws, approves, supplies):
#     print(f"TRANSACTION {index}")
#     index += 1

#     for (data, name) in zip((withdraw, approve, supply), ("Withdraw", "Approve", "Supply")):
#         print(f"{name}:")

#         if not data:
#             print("<EMPTY>")
#             continue

#         (params, receipt) = data

#         print(f"Transaction parameters:")
#         pprint(params)
#         print(f"Transaction receipt:")
#         pprint(dict(receipt))
    
#     print()

withdraws

[None,
 ({'value': 0,
   'gas': 84630,
   'maxFeePerGas': 26299765125,
   'maxPriorityFeePerGas': 26299765079,
   'chainId': 137,
   'from': '0x90259C703a645e235B22572CC9a45a1b8977887c',
   'nonce': 891,
   'to': '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
   'data': '0x095ea7b3000000000000000000000000794a61358d6845594f94dc1db02a252b5b4814adffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'},
  AttributeDict({'blockHash': HexBytes('0xc5062f136b98bca233d4900eb234374d225406bc03df0bfd6ebebc35ec5953c2'),
   'blockNumber': 62070800,
   'contractAddress': None,
   'cumulativeGasUsed': 8715335,
   'effectiveGasPrice': 26299765104,
   'from': '0x90259C703a645e235B22572CC9a45a1b8977887c',
   'gasUsed': 34972,
   'logs': [AttributeDict({'address': '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
     'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'),
      HexBytes('0x00000000000000000000000090259c703a645e235b22572cc9a45a1b8977887c'),
      H

In [5]:
# From Discord - similar errors

tx = {
        'from': account.address,
        'value': 0,
        'chainId': await w3.eth.chain_id,
        'gas': 250000,
        'maxFeePerGas': await w3.eth.gas_price * 2,
        'maxPriorityFeePerGas': await w3.eth.max_priority_fee * 2,
}

def sign_tx(tx, key):
  return w3.eth.account.sign_transaction(tx, private_key=key)

async def send_tx(signed_tx):
  return await w3.eth.send_raw_transaction(signed_tx.raw_transaction)

async def main():
  amount    = 1 * 10 **15
  approve   = token_contract.functions.approve(pool_address, amount)
  supply    = pool_contract.functions.supply(token_address, amount, account.address, 0)
  withdraw  = pool_contract.functions.withdraw(token_address, amount, account.address)

  tx.update({'nonce': await w3.eth.get_transaction_count(account.address)})

  approve = await approve.build_transaction(tx)
  print ('[-] Approving... ')
  tx_hash = await send_tx(sign_tx(approve, account.key))
  receipt = await w3.eth.wait_for_transaction_receipt(tx_hash)
  print (f'[+] Approved at TOKEN contract: {tx_hash.hex()}\n[>] {receipt}')

  tx.update({'nonce': await w3.eth.get_transaction_count(account.address)})

  supply  = await supply.build_transaction(tx)
  print('[-] Simulating supply...')
  await w3.eth.call(supply)
  print ('[-] Supplying... ')

  tx_hash = await send_tx(sign_tx(supply, account.key))
  print(f"[+] Supply went through: {tx_hash.hex()}")
  receipt = await w3.eth.wait_for_transaction_receipt(tx_hash)
  print (f'[+] Supply complete:\n[>]{receipt}')

  tx.update({'nonce': await w3.eth.get_transaction_count(account.address)})

  withdraw  = await withdraw.build_transaction(tx)
  print('[-] Simulating withdraw...')
  await w3.eth.call(withdraw)
  print('[-] Attempting withdraw...')
  tx_hash = await send_tx(sign_tx(withdraw, account.key))
  print(f"[+] Withdraw went through: {tx_hash.hex()}")
  receipt = await w3.eth.wait_for_transaction_receipt(tx_hash)
  print (f'[>] Withdraw complete:\n[>]{receipt}')

while True:
  await main()


[-] Approving... 
[+] Approved at TOKEN contract: b'\x89\xd25j,\xf3\xf0\xfd\x99\x19w\xeb\xc2X\xe8\xf3q\xe9\x88n\xc0d\xbd\x8a\x9b\xf0\x87V\x91\xb1\xb0\x01'
[>] AttributeDict({'blockHash': HexBytes('0xe35ac9d72a0dd0f50d2b982ce93308f76540abc72ceceec32802d122c9e30a61'), 'blockNumber': 62062390, 'contractAddress': None, 'cumulativeGasUsed': 3105372, 'effectiveGasPrice': 88624013234, 'from': '0x90259C703a645e235B22572CC9a45a1b8977887c', 'gasUsed': 51760, 'logs': [AttributeDict({'address': '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063', 'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'), HexBytes('0x00000000000000000000000090259c703a645e235b22572cc9a45a1b8977887c'), HexBytes('0x000000000000000000000000794a61358d6845594f94dc1db02a252b5b4814ad')], 'data': HexBytes('0x00000000000000000000000000000000000000000000000000038d7ea4c68000'), 'blockNumber': 62062390, 'transactionHash': HexBytes('0x89d2356a2cf3f0fd991977ebc258e8f371e9886ec064bd8a9bf0875691b1b001'), 't

ContractLogicError: ('execution reverted: 32', '0x08c379a0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000023332000000000000000000000000000000000000000000000000000000000000')